In [1]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import random

In [2]:
dic = {
    'source_id':[],
    'manual_id':[],
    'article_title':[],
    'url':[],
    'authors':[],
    'author_affiliation':[],
    'abstract_text':[],
    'date':[],
    'start_time':[],
    'end_time':[],
    'location':[],
    'session_title':[],
    'session_type':[],
    'category':[],
    'sub_category':[],
    'disclosure':[]
    }

In [6]:
req1 = requests.get("https://www.eventscribe.net/2022/nacfcprogram/agenda.asp?pfp=Browse%20by%20Day")
soup = BeautifulSoup(req1.content, 'html.parser')

In [7]:
c = 0
for i in soup.find_all(class_=re.compile('list-group-item loadbyurl.*'))[0:]:
    code = i.get('data-presid')
    icon = ''
    if i.find_all('img'): # check if there is any icon
        print('yesss iconn......')
        for i in i.find_all('img'):
            icon += i.get('title') + '; '
    
    session_url = f'https://www.eventscribe.net/2022/nacfcprogram/fsPopup.asp?efp=Vk9PSkdNRFExNjM2Ng&PresentationID={code}&rnd=0.7055475&mode=presinfo'
    req = requests.get(session_url)
    soup = BeautifulSoup(req.content, 'html.parser')

    print(session_url)
    session_title = soup.find('h1').text.strip()
    session_date = soup.find(class_='pull-left pres-tidbit tipsytip').text.strip()
    ses_time_loc = soup.find_all(class_='pull-left pres-tidbit')
    session_time = ses_time_loc[0].text.strip()
    
    if len(ses_time_loc) > 1:
        session_loc = ses_time_loc[1].text.strip()
    else:
        print('no location....')
        session_loc = ''
        
    sess_text = ''
    session_author = ''
    affiliation = ''
    sess_type = ''
    category = ''
    cat = soup.find_all(class_='trackname innertracks')
    if len(cat) < 1:
        cat = soup.find_all(style='color:#FFFFFF;font-size:14px; margin-top:10px; margin-right:10px; background:#489CD8')
    
    try:
        sess_type = soup.find(style='color:#FFFFFF; font-size:14px; margin-top:10px; margin-right:10px; background:#489CD8;').text.strip()
    except:
        try:
            sess_type = soup.find(style='margin-top:10px;').text.strip()
        except:
            sess_type = ''

    for i in cat:
        category += i.text.strip() + '; '

    auth = soup.find(class_='speakers-wrap')

    for s_i in auth.find_all(class_='row'): # author affiliation
        aut = s_i.find(class_='col-xs-12 col-md-6').a.text
        aff = s_i.find(class_="text-muted prof-text")
        aff = re.split('<br.*?>', str(aff))
        aff = ' '.join(aff)
        if re.search('<p class="text-muted prof-text">(.*?)</p>', aff):
            aff = re.search('<p class="text-muted prof-text">(.*?)</p>', aff).group(1)
        else:
            aff = ' '
        session_author += aut.strip() + '; '
        affiliation += aff + '; '
        
    try: # abs_text with giving spaces
        sess_text = soup.find(class_='PresentationAbstractText mar-top')
        sess_text = str(sess_text)
        sess_text = re.sub('<.*?>', ' ', sess_text)
    except:
        sess_text = ''
    if sess_text == None:
        sess_text = ''

    if session_title == sess_type:
        sess_type = ' '
        print('check if.......')
        
    category = re.sub(sess_type+'; ', '', category)
    print(f'session title :- {session_title}')
    print(f'session type :- {sess_type}')
    print(f'category :- {category}')
    print(f'session date :- {session_date}')
    print(f'session time :- {session_time}')
    print(f'session location :- {session_loc}')
    print(f'session author :- {session_author}')
    print(f'session aff :- {affiliation}')
    print(f'session text :- {sess_text}')

    print('======================='*5)
    
    dic['session_number'].append(session_number)
    dic['name'].append('Session')
    dic['source_id'].append('')
    dic['manual_id'].append('')
    dic['article_title'].append(session_title)
    dic['url'].append(session_url)
    dic['authors'].append(session_author)
    dic['author_affiliation'].append(affiliation)
    dic['abstract_text'].append(sess_text)
    dic['date'].append(session_date)
    dic['start_time'].append(session_time)
    dic['end_time'].append('')
    dic['location'].append(session_loc)
    dic['session_title'].append(session_title)
    dic['session_type'].append(sess_type)
    dic['category'].append(category)
    dic['sub_category'].append('')
    dic['disclosure'].append(icon)
    
    
    for i in re.split("li class='list-group-item '", req.text)[1:]: # article links

        url = re.search("data-presid='(.*?)' data-url", i).group(1)
        article_url = f'https://www.eventscribe.net/2022/nacfcprogram/fsPopup.asp?Mode=presInfo&PresentationID={url}'
        print(article_url)
        req_art = requests.get(article_url)
        soup = BeautifulSoup(req_art.content, 'html.parser')
        
        article_cat = category
        cat = soup.find_all(style='background:#489CD8; color:#FFFFFF; font-size:14px; margin-top:10px; margin-right:10px;')
        if len(cat) < 1:
            cat = soup.find_all(style='color:#FFFFFF;font-size:14px; margin-top:10px; margin-right:10px; background:#489CD8')
        
        for i in cat:
            article_cat += i.text.strip() + '; '
            
        article_title = soup.find('h1', style='margin-top:10px;').text.strip()
        article_date = soup.find('div', class_='pull-left pres-tidbit tipsytip').text.strip()
        article_author = ''
        article_aff = ''
        article_abs_text = ''
        time_loc = soup.find_all(class_='pull-left pres-tidbit')
        art_time = time_loc[0].text.strip()
        art_auth = soup.find(class_='speakers-wrap')

        for a_i in art_auth.find_all(class_='row'): # author affiliation
            aut = a_i.find(class_='col-xs-12 col-md-6').a.text
            aff = a_i.find(class_="text-muted prof-text")
            aff = re.split('<br.*?>', str(aff))
            aff = ' '.join(aff)
            if re.search('<p class="text-muted prof-text">(.*?)</p>', aff):
                aff = re.search('<p class="text-muted prof-text">(.*?)</p>', aff).group(1)
            else:
                aff = ' '
            article_author += aut.strip() + '; '
            article_aff += aff + '; '
        try:
            article_abs_text = soup.find(class_='PresentationAbstractText mar-top')
            article_abs_text = str(article_abs_text)
            article_abs_text = article_abs_text.replace('<b>', '')
            article_abs_text = article_abs_text.replace('</b>', '')
            article_abs_text = re.split('<br.*?>', article_abs_text)
            article_abs_text = ' '.join(article_abs_text)
            article_abs_text = re.search('<div class="PresentationAbstractText mar-top">(.*?)</div>', article_abs_text).group(1)
        except:
            article_abs_text = ''

        print(article_title)
        print(article_date)
        print(art_time)
        print(article_author)
        print(article_aff)
        print(f'article text :- {article_abs_text}')
        print(f'dusra column :- {article_cat}')
        
        dic['session_number'].append(session_number)
        dic['name'].append('Presentation')
        dic['source_id'].append('')
        dic['manual_id'].append('')
        dic['article_title'].append(article_title)
        dic['url'].append(article_url)
        dic['authors'].append(article_author)
        dic['author_affiliation'].append(article_aff)
        dic['abstract_text'].append(article_abs_text)
        dic['date'].append(article_date)
        dic['start_time'].append(art_time)
        dic['end_time'].append('')
        dic['location'].append(session_loc)
        dic['session_title'].append(session_title)
        dic['session_type'].append(sess_type)
        dic['category'].append(category)
        dic['sub_category'].append('')
        dic['disclosure'].append('')
       
    c += 1
    print(c)

https://www.eventscribe.net/2022/nacfcprogram/fsPopup.asp?efp=Vk9PSkdNRFExNjM2Ng&PresentationID=1148840&rnd=0.7055475&mode=presinfo
session title :- Mental Health Advisory Committee Meeting
session type :- Affiliate Meeting
category :- 
session date :- Tuesday, November 1, 2022
session time :- 5:00 PM – 8:00 PM ET
session location :- Location: Marriott Downtown - Room 303
session author :- 
session aff :- 
session text :-  Invite Only 
1
https://www.eventscribe.net/2022/nacfcprogram/fsPopup.asp?efp=Vk9PSkdNRFExNjM2Ng&PresentationID=1148843&rnd=0.7055475&mode=presinfo
session title :- RARE Study Meeting
session type :- Affiliate Meeting
category :- 
session date :- Wednesday, November 2, 2022
session time :- 7:00 AM – 8:30 AM ET
session location :- Location: Marriott Downtown - Room 301
session author :- 
session aff :- 
session text :-  Invite Only 
2
https://www.eventscribe.net/2022/nacfcprogram/fsPopup.asp?efp=Vk9PSkdNRFExNjM2Ng&PresentationID=1087159&rnd=0.7055475&mode=presinfo
sess


KeyboardInterrupt



In [15]:
df = pd.DataFrame(dic)
df

,source_id,manual_id,article_title,url,authors,author_affiliation,abstract_text,date,start_time,end_time,location,session_title,session_type,category,sub_category,disclosure
0,,,Mental Health Advisory Committee Meeting,https://www.eventscribe.net/2022/nacfcprogram/...,,,Invite Only,"Tuesday, November 1, 2022",5:00 PM – 8:00 PM ET,,Location: Marriott Downtown - Room 303,Mental Health Advisory Committee Meeting,Affiliate Meeting,,,
1,,,RARE Study Meeting,https://www.eventscribe.net/2022/nacfcprogram/...,,,Invite Only,"Wednesday, November 2, 2022",7:00 AM – 8:30 AM ET,,Location: Marriott Downtown - Room 301,RARE Study Meeting,Affiliate Meeting,,,
2,,,PEP - Partnership Enhancement Program (PEP) Wo...,https://www.eventscribe.net/2022/nacfcprogram/...,"Karen M. von Berg, PT, DPT; Elizabeth Bryson, ...",Johns Hopkins Hospital; Akron Children's Hospi...,"Establishing strong, trusting relationships i...","Wednesday, November 2, 2022",8:00 AM – 3:00 PM ET,,Location: 103 C,PEP - Partnership Enhancement Program (PEP) Wo...,PEP,,,
3,,,SPC01 - Basic Airway Clearance Therapy Course,https://www.eventscribe.net/2022/nacfcprogram/...,"Maggie McIlwaine, PhD; Brenda M. Button, PhD; ...",University of British Columbia; Dept. Medicine...,This course is primarily for Respiratory and ...,"Wednesday, November 2, 2022",8:00 AM – 5:00 PM ET,,Location: 204 AB,SPC01 - Basic Airway Clearance Therapy Course,Special Class,PT&RT;,,
4,,,SPC02 - Nuts & Bolts for the New CF Team Member,https://www.eventscribe.net/2022/nacfcprogram/...,"Lynn Feenan, RN, MS; Emily K. Seamans, PharmD,...",New Hampshire CF Center; Dartmouth-Hitchcock M...,This day-long special class is designed for n...,"Wednesday, November 2, 2022",8:00 AM – 5:00 PM ET,,Location: 122 AB,SPC02 - Nuts & Bolts for the New CF Team Member,Special Class,NRS; CLIN; SW/PSYCH;,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,,,W37.2- Proteomic analysis identifies physiolog...,https://www.eventscribe.net/2022/nacfcprogram/...,"Emily J. Skala, BS;",Cincinnati Children's Hospital Medical Center;,,"Saturday, November 5, 2022",3:30 PM – 5:30 PM ET,,Location:108 AB,W37--Biomarkers,Workshop,APPTT; CLIN;,,
867,,,W37.3- Elexacaftor/Tezacaftor/Ivacaftor treatm...,https://www.eventscribe.net/2022/nacfcprogram/...,"Nicola Robinson, MBChB(Hons) MRCPUK;","Centre for Inflammation Research, University o...",,"Saturday, November 5, 2022",3:30 PM – 5:30 PM ET,,Location:108 AB,W37--Biomarkers,Workshop,APPTT; CLIN;,,
868,,,W37.4- Change in liver biomarkers after introd...,https://www.eventscribe.net/2022/nacfcprogram/...,"Christine Højte, MD;","Copenhagen Cystic Fibrosis Center, Rigshospita...",,"Saturday, November 5, 2022",3:30 PM – 5:30 PM ET,,Location:108 AB,W37--Biomarkers,Workshop,APPTT; CLIN;,,
869,,,W37.5- Specific respiratory conductance by for...,https://www.eventscribe.net/2022/nacfcprogram/...,"Elpis Hatziagorou, PhD, MD;",Aristotle University of Thessaloniki;,,"Saturday, November 5, 2022",3:30 PM – 5:30 PM ET,,Location:108 AB,W37--Biomarkers,Workshop,APPTT; CLIN;,,


In [16]:
df.to_excel('nacfc_.xlsx')